In [1]:
import pandas as pd
import numpy as np
import scipy as sp
from scipy.misc import comb
from sklearn.model_selection import train_test_split , KFold, StratifiedKFold
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.preprocessing import normalize
import pdb
from sklearn.metrics import roc_auc_score , make_scorer
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import NuestroArbol as ourTree
import random
from scipy.stats import randint as sp_randint
from time import time
import collections
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from IPython.core.display import HTML



%matplotlib inline

# Importo datos 

In [2]:
X_comp = pd.read_csv('X_competencia.csv')
X = pd.read_csv('X.csv')
y = pd.read_csv('y.csv')
X.drop(['index'],inplace=True, axis=1)
y.drop(['index'], inplace=True,axis=1)



In [3]:
#partimos los datos en desarrollo(87%) y holdout(13%). 
X_desarrollo , X_holout ,y_desarrollo, y_holdout = train_test_split(X, y['output'],
                                                                    test_size=0.13,random_state=0,stratify=y['output'])

#a = normalize(X_desarrollo, axis=0, copy=True, return_norm=False)
#from IPython.core.debugger import Tracer; Tracer()() 
#X_desarrollo = pd.DataFrame(SelectKBest(chi2, k=180).fit_transform(np.abs(X_desarrollo), y_desarrollo))

#pase los kfold aca arriba ya que se usan en varios lugares.
#evaluar si esta bueno que siempre se usen los mismos folds
kfold = StratifiedKFold(n_splits=5)
kfold.get_n_splits(X_desarrollo,y_desarrollo)


5

# Gaussian NB

In [6]:
from sklearn.naive_bayes import GaussianNB



def accuracyForTrainingAndValidation():
    ROC_AUC_train=[]
    ROC_AUC_validation=[]

    #este for itera sobre los k folds en cada loop tego un set de training y otro de validacion
    for train, test  in kfold.split(X_desarrollo,y_desarrollo):
        #print("TRAIN:", train_index,'\n', "TEST:", test_index,'\n' )
        X_train, X_val = X_desarrollo.iloc[train], X_desarrollo.iloc[test]
        y_train, y_val = y_desarrollo.iloc[train], y_desarrollo.iloc[test]
        #intancio el arbol que voy a entrenar en cada fold
        gnb = GaussianNB()


        gnb.fit(X_train, y_train.astype(int))
        ROC_AUC_train.append(roc_auc_score(y_train,gnb.predict(X_train)))
        ROC_AUC_validation.append(roc_auc_score(y_val,gnb.predict(X_val)))
        
    return pd.DataFrame({"training":[np.mean(ROC_AUC_train)], "validation": [np.mean(ROC_AUC_validation)]})

        

# NB

In [7]:
display(accuracyForTrainingAndValidation()) 

,training,validation
0,0.766185,0.716033
